## **Intro**

* Julia is more powerful than python when it comes to manipulating arrays more than 3 dimensions. 


### **Initialization** 

|Function              |	Description|
|----------------------|-------------|
|Array{T}(undef, dims...) |	an uninitialized dense Array|
| zeros(T, dims...)	| an Array of all zeros |
| ones(T, dims...)	| an Array of all ones |
| trues(dims...)	| a BitArray with all values true | 
| falses(dims...)   | 	a BitArray with all values false | 
| reshape(A, dims...)	|an array containing the same data as A, but with different dimensions|
copy(A)	| copy A |
| deepcopy(A)|	copy A, recursively copying its elements|
| similar(A, T, dims...)	|an uninitialized array of the same type as A (dense, sparse, etc.),but with the specified element type and dimensions. The second and third arguments are both optional, defaulting to the element type and dimensions of A if omitted.|
| reinterpret(T, A)	|an array with the same binary data as A, but with element type T|
| rand(T, dims...) |	an Array with random, iid [1] and uniformly distributed values in the half-open interval [0, 1)[0,1)|
| randn(T, dims...)	| an Array with random, iid and standard normally distributed values |
|Matrix{T}(I, m, n)	| m-by-n identity matrix. Requires using LinearAlgebra for I. |
| range(start, stop=stop, length=n)	| range of n linearly spaced elements from start to stop|
| fill!(A, x)	| fill the array A with the value x|
| fill(x, dims...)	|an Array filled with the value x|



In [2]:
# let's initialize some data. 
A = zeros(Int64, 3, 4)
B = reinterpret(Float64, A)


3×4 reinterpret(Float64, ::Matrix{Int64}):
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [11]:
display(range(1, 2, length=100))
# stepping is not going to include both end. 
display(range(0, 1, step=0.1))
# the returned type is an iterator, so this is like, and it's inclusive at both end. 
(display∘collect)(0:0.1:1)

1.0:0.010101010101010102:2.0

0.0:0.1:1.0

11-element Vector{Float64}:
 0.0
 0.1
 0.2
 0.3
 0.4
 0.5
 0.6
 0.7
 0.8
 0.9
 1.0

In [35]:
A = Matrix{Int64}(undef, 3, 3)
A[begin:end] .= 3
display(A)
B = fill(4, (3, 3))
display(B)
D = fill("Brugh", (3, 3))
display(C)

3×3 Matrix{Int64}:
 3  3  3
 3  3  3
 3  3  3

3×3 Matrix{Int64}:
 4  4  4
 4  4  4
 4  4  4

3×3 Matrix{String}:
 "Brugh"  "Brugh"  "Brugh"
 "Brugh"  "Brugh"  "Brugh"
 "Brugh"  "Brugh"  "Brugh"

### **Basic Manipulations**

|Function	| Description| 
|----------|--------------|
| eltype(A)	    | the type of the elements contained in A  | 
| length(A)	    | the number of elements in A  | 
| ndims(A)	    | the number of dimensions of A  | 
| size(A)	    | a tuple containing the dimensions of A  | 
| size(A,n)	    | the size of A along dimension n  | 
| axes(A)	    | a tuple containing the valid indices of A  | 
| axes(A,n)	    | a range expressing the valid indices along dimension n  | 
| eachindex(A)	| an efficient iterator for visiting each position in A  | 
| stride(A,k)	| the stride (linear index distance between adjacent elements) along dimension k  | 
| strides(A)	| a tuple of the strides in each dimension  | 

In [2]:
A = fill(0, (3, 3))
display(eltype(A))
display(length(A))

Int64

9

In [27]:
A = fill(0, (3, 4, 5))
axis1 = (collect ∘ axes)(A)
for II = axis1[1], JJ = axis1[2]
    (println ∘ string)(II, " ",JJ)
end


1 1
1 2
1 3
1 4
2 1
2 2
2 3
2 4
3 1
3 2
3 3
3 4


**Stride**
Say, I have `m by n by k` tensor, to stride across the first dimension, we make linear index with distance `m`, to strice across different pages, we stride across `m by n` elements in the linear index. 



In [49]:
A = zeros(2, 3, 4)
display(stride(A, 2))
display(strides(A))
# Check out the second fibre of the tensor: 
Indices = []
for II in 1: stride(A, 2): length(A)
    append!(Indices, II)
end
length(Indices) 
# 3 x 4, we are fixing the first element and then stride through each indices for the second 2 dimensions. 

2

(1, 2, 6)

12

### **Fancier Tensors Reductions, BoardCasting**
* Map
* Filter
* Reduce
  * Reduce the tensor along a certain dimension, with a binary operator.
  * If the operator has left/right associated property, use `foldr`  and `foldl` instead. 
  * Reduction might accumulate errors. It's a linear reduction of some kind. 


* `map`, `map!`, `mapfoldr`, `mapfoldl`, `mapslices`, `mapreduce`, `asyncmap`, `asyncmap!`
* Map Reduce
  * Function, binary reduction, and along certain dimension of the tensor. 

In [1]:
map((x)->x^2, 1:10)

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [3]:
f(x) = x^2 + 1
A = rand(3,3)
display(A)
map(f, A)

3×3 Matrix{Float64}:
 0.778609  0.330053  0.189242
 0.545945  0.123249  0.522528
 0.12579   0.285666  0.204248

3×3 Matrix{Float64}:
 1.60623  1.10893  1.03581
 1.29806  1.01519  1.27304
 1.01582  1.08161  1.04172

In [9]:
# inner product in disguise. 
sum(map(*, [1, 2, 3], [1, 2, 3]))

14

In [8]:
x = [1, 2, 3]
display(x)
reduce((x, y) -> abs(x - y), x)

3-element Vector{Int64}:
 1
 2
 3

2

In [4]:
# map reduce 
mapreduce(x -> x^2, +, [1 2 3; 4 5 6], dims=1)

1×3 Matrix{Int64}:
 17  29  45

In [5]:
# Filter out elements from a tensor
out = filter(isodd, [1 2 3; 4 5 6])
display(out)

3-element Vector{Int64}:
 1
 5
 3

### **Common Reductions Techniques**
1. maximum
2. minimum
3. sum
4. prod
5. any 
6. all

### **Linear Algebra**

